# Goal of this notebook

Develop a training loop for finetuning ASR models using TTS loss by recreating RL training found in RL4LMs/rl4lms/envs/text_generation/training_utils.py

```bash
conda activate speller
cd /home/s1785140/rlspeller
```

# automatic reloading magic

In [1]:
%load_ext autoreload
%autoreload 2

# imports

In [34]:
import torch
from typing import List, Dict, Tuple, Any
import hyperpyyaml
from tqdm import tqdm
from torchaudio.models.decoder import ctc_decoder
from torch.nn.functional import softmax
import random
from jiwer import cer
import numpy as np
import speechbrain as sb
from IPython.display import Audio
import matplotlib.pyplot as plt
import librosa
import pandas as pd

## check if gpu available

In [3]:
# print hostname to make sure we are on correct node
disallowed_nodes = ['escience6']
import socket
hostname = socket.gethostname()
print(hostname)
node = hostname.split('.')[0]
if node in disallowed_nodes:
    raise ValueError(f"Running on disallowed node {node}!")

greider.inf.ed.ac.uk


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()
assert torch.cuda.is_available()

In [5]:
import os
os.getcwd()

'/disk/nfs/ostrom/s1785140/rlspeller'

# HPARAMS

In [6]:
inference_hparams = {
    "sentencepiece_model_path": "/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/Tokenizer/save/0_char.model",

    # 'speechbrain_hparams_file': '/home/s1785140/rlspeller/infer_speechbrain_subsampling1x_nowhitespace.yaml',
    'speechbrain_hparams_file': '/home/s1785140/rlspeller/infer_speechbrain_subsampling1x.yaml',
    # 'speechbrain_hparams_file': '/home/s1785140/rlspeller/infer_speechbrain_subsampling2x.yaml',
    # 'speechbrain_hparams_file': '/home/s1785140/rlspeller/infer_speechbrain.yaml', # 4x subsampling

    'sample_rate': 16000,
    
    'enable_dropout': True,
    # 'dropout_p': 0.0, # 0.15 is the default value in speechbrain
    'dropout_p': 0.15, # 0.15 is the default value in speechbrain
    # 'dropout_p': 0.3, # 0.15 is the default value in speechbrain
    # 'dropout_p': 0.5, # 0.15 is the default value in speechbrain
    # 'dropout_p': 0.99, # 0.15 is the default value in speechbrain
}

# TOKENIZER

In [7]:
# load pretrained tokenizer used to tokenizer ASR training inputs 
import sentencepiece as spm 
spm_path = inference_hparams["sentencepiece_model_path"]
sp = spm.SentencePieceProcessor()
sp.load(spm_path)
print(sp.vocab_size())

28


In [8]:
# test tokenizer
s = "hello world my name is jason"
# TODO pass string through text cleaners? 
encoded = sp.EncodeAsIds(s)
assert 0 not in encoded, "tried to encode an unknown character"
print(" ".join(str(idx) for idx in encoded))

1 10 2 12 12 4 1 17 4 9 12 11 1 16 20 1 6 5 16 2 1 7 8 1 26 5 8 4 6


In [9]:
sp.DecodeIds(encoded)

'hello world my name is jason'

# NEW! SIMPLE TOKENIZER

In [10]:
from speechbrain.tokenizers.SimpleTokenizer import SimpleTokenizer

In [11]:
tokenizer = SimpleTokenizer()

In [12]:
text = "hello my name is jason"
text = text.replace(' ', '|')
print(text)
ids = tokenizer.encode_as_ids(text)
ids

hello|my|name|is|jason


[9, 6, 13, 13, 16, 1, 14, 26, 1, 15, 2, 14, 6, 1, 10, 20, 1, 11, 2, 20, 16, 15]

In [13]:
tokenizer.decode_ids(ids)

'hello|my|name|is|jason'

## test simple tokenizer with probability distribution, and see if CTC decoder successfully generates n-best lists

In [14]:
# create empty array of correct dimensions
min_len, max_len = 50, 100
bsz = 4
lens = torch.randint(min_len, max_len, (bsz,))
vocab_size = len(tokenizer.vocab)

# randomly assign probaility distribution to each timestep

# try to decode

In [15]:
randn = torch.randn(bsz, max_len, vocab_size)

In [16]:
ctc_probs = softmax(randn, dim=1)
# ctc_probs

In [17]:
ctc_beamsearch_decoder_test = ctc_decoder(
    lexicon=None,
    # tokens="/home/s1785140/rlspeller/templates/speech_recognition_CharTokens_NoLM/Tokenizer/save/tokens.txt",
    tokens=tokenizer.vocab,
    nbest=2,
    blank_token='-',
    sil_token="|",
)

predicted_ids = ctc_beamsearch_decoder_test(ctc_probs, lens)

predicted_words = []
for i, hyps in enumerate(predicted_ids):
    for j, hyp in enumerate(hyps):
        words = tokenizer.decode_ids(hyp.tokens.tolist()).split(" ")
        tup = (f"sample {i+1}, hyp {j+1}/{len(hyps)}", words)
        predicted_words.append(tup)
        print(tup)

('sample 1, hyp 1/2', ['|bxjwjusxlgtnhngvkceu|yokslkiqjfqsbtqucpmqix|vyxfrerjkbswofpcixlzrpza|nlv|jvcarktm|'])
('sample 1, hyp 2/2', ['|bxjwjusxlgtnhngvkceu|yokslkiqjfqsbtqucpmqix|vyxfregjkbswofpcixlzrpza|nlv|jvcarktm|'])
('sample 2, hyp 1/2', ['|bam|glpxiuvoc|vgynytgxsknhctfogdncsnck|rnmlovot|'])
('sample 2, hyp 2/2', ['|bam|glpxiuvoc|vgynytgxsknwctfogdncsnck|rnmlovot|'])
('sample 3, hyp 1/2', ['|trbiukvjvljqfdaspbxbrmd|reolrgbgzbftpcgcibsnmwkyqmhtshts|'])
('sample 3, hyp 2/2', ['|trbiukvjvljqfdaspbxbrmd|reolrgbgzbftpcgcibsnmwkyqmheshts|'])
('sample 4, hyp 1/2', ['|cdlgtrygvsxqfuyulfstgbznomg|jklzelnjquendebviwapvxckrmcqshf|'])
('sample 4, hyp 2/2', ['|cdlgtrygvsxqfuyulfstgbcnomg|jklzelnjquendebviwapvxckrmcqshf|'])


# LOAD ASR (PRETRAINED)

In [18]:
from templates.speech_recognition_CharTokens_NoLM.ASR.train import ASR
from templates.speech_recognition_CharTokens_NoLM.ASR.train import dataio_prepare
from torch.utils.data import DataLoader
from speechbrain.dataio.dataloader import LoopedLoader

In [19]:
# Load hyperparameters file with command-line overrides
speechbrain_hparams_file = inference_hparams['speechbrain_hparams_file']
with open(speechbrain_hparams_file) as f:
    speechbrain_hparams = hyperpyyaml.load_hyperpyyaml(f)

/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/data/rirs_noises.zip exists. Skipping download


In [20]:
speechbrain_hparams['save_folder']

'/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/ASR/results/CRDNN_CHAR_LJSpeech_halved_subsampling1x/2602/save'

In [21]:
# replace some values with inference time values
speechbrain_hparams['dropout'] = inference_hparams['dropout_p']

In [22]:
# initialise trainer (we don't want to train, but model is tightly coupled with trainer)
asr_brain = ASR(
    modules=speechbrain_hparams["modules"],
    opt_class=speechbrain_hparams["opt_class"],
    hparams=speechbrain_hparams,
    checkpointer=speechbrain_hparams["checkpointer"],
)

In [23]:
def change_dropout_p(brain, new_dropout_p):
    for module_name in brain.modules.keys():
        module = brain.modules[module_name]
        for submodule_name, submodule in module.named_modules():
            if isinstance(submodule, torch.nn.Dropout) or isinstance(submodule, torch.nn.Dropout2d):
                # handle proper dropout submodules
                print(f"changed dropout p of {module_name}-{submodule_name}-{submodule} from {submodule.p} to {new_dropout_p}")
                submodule.p = new_dropout_p
            else:
                # handle dropout in nn.LSTM
                for attr in ["dropout", "drop"]:
                    if hasattr(submodule, attr) and type(getattr(submodule, attr)) == float: 
                        print(f"changed dropout p of {module_name}-{submodule_name}-{submodule} from {getattr(submodule, attr)} to {new_dropout_p}")
                        setattr(submodule, attr, new_dropout_p)

def setup_asr_brain_for_infer(asr_brain, enable_dropout=False, new_dropout_p=0.15):
    asr_brain.on_evaluate_start(min_key="WER") # We call the on_evaluate_start that will load the best model
    if enable_dropout:
        asr_brain.modules.train()
        print("Enabled dropout in model:")
        change_dropout_p(asr_brain, new_dropout_p)
    else:
        asr_brain.modules.eval() # We set the model to eval mode (remove dropout etc)

print("if on_evaluate_start() get runtime error, likely need to restart notebook kernel")
setup_asr_brain_for_infer(asr_brain, enable_dropout=inference_hparams["enable_dropout"], new_dropout_p=inference_hparams["dropout_p"])

if on_evaluate_start() get runtime error, likely need to restart notebook kernel
Enabled dropout in model:
changed dropout p of encoder-CNN.block_0.drop.drop-Dropout2d(p=0.15, inplace=False) from 0.15 to 0.15
changed dropout p of encoder-CNN.block_1.drop.drop-Dropout2d(p=0.15, inplace=False) from 0.15 to 0.15
changed dropout p of encoder-RNN.rnn-LSTM(2560, 512, num_layers=2, batch_first=True, dropout=0.15, bidirectional=True) from 0.15 to 0.15
changed dropout p of encoder-DNN.block_0.dropout-Dropout(p=0.15, inplace=False) from 0.15 to 0.15
changed dropout p of encoder-DNN.block_1.dropout-Dropout(p=0.15, inplace=False) from 0.15 to 0.15


In [24]:
# create dataset and dataloader for inference
datasets = dataio_prepare(speechbrain_hparams)

test_set = datasets['test']

if not isinstance(test_set, DataLoader) or isinstance(test_set, LoopedLoader):
    test_loader_kwargs=speechbrain_hparams["test_dataloader_opts"]
    test_set = asr_brain.make_dataloader(
        test_set, stage=sb.Stage.TEST, **test_loader_kwargs
    )

In [25]:
# get vocab from tokenizer (needed for ctc decoding)
vocab_size = len(asr_brain.hparams.tokenizer)
vocab = []
for i in range(vocab_size):
    vocab.append(asr_brain.hparams.tokenizer.decode_ids([i]))
print(vocab)

# edit vocab to match default ctc decoder symbols for blank and silence
vocab[0] = '-'
vocab[1] = "|"

print(vocab)

[' ⁇ ', '', 'e', 't', 'o', 'a', 'n', 'i', 's', 'r', 'h', 'd', 'l', 'c', 'f', 'u', 'm', 'w', 'p', 'g', 'y', 'b', 'v', 'k', 'x', 'q', 'j', 'z']
['-', '|', 'e', 't', 'o', 'a', 'n', 'i', 's', 'r', 'h', 'd', 'l', 'c', 'f', 'u', 'm', 'w', 'p', 'g', 'y', 'b', 'v', 'k', 'x', 'q', 'j', 'z']


In [26]:
ctc_beamsearch_decoder = ctc_decoder(
    lexicon=None,
    # tokens="/home/s1785140/rlspeller/templates/speech_recognition_CharTokens_NoLM/Tokenizer/save/tokens.txt",
    tokens=vocab,
    nbest=100,
    blank_token='-',
    sil_token="|",
)

In [27]:
# generate transcriptions for all batches in test set
def transcribe_dataset(asr_brain, dataset, greedy=False, num_batches_to_transcribe=None):
    # Now we iterate over the dataset and we simply compute_forward and decode
    with torch.no_grad():
        transcripts = []
        for batch in tqdm(list(dataset)[:num_batches_to_transcribe], dynamic_ncols=True):
            orig_transcriptions = batch.words

            # Make sure that your compute_forward returns the predictions !!!
            # In the case of the template, when stage = TEST, a beam search is applied 
            # in compute_forward(). 
            predictions = asr_brain.compute_forward(batch, stage=sb.Stage.TEST)
            
            ctc_probs = predictions['ctc_logprobs'] # FOR DEBUG

            if greedy:
                predicted_ids = sb.decoders.ctc_greedy_decode(
                    predictions["ctc_logprobs"], asr_brain.feat_lens, blank_id=asr_brain.hparams.blank_index
                )
                predicted_words = [
                    asr_brain.tokenizer.decode_ids(ids).split(" ")
                    for ids in predicted_ids
                ]
            else:
                # get mel lens from wav len ratios since torch ctc decoder requires lens in frames
                batch_max_len = predictions["ctc_logprobs"].size(1)
                bsz = predictions["ctc_logprobs"].size(0)
                mel_lens = torch.zeros(bsz)
                for i, len_ratio in enumerate(asr_brain.feat_lens):
                    mel_lens[i] = int(torch.round(len_ratio * batch_max_len))
                
                predicted_ids = ctc_beamsearch_decoder(
                    predictions["ctc_logprobs"], lengths=mel_lens
                )

                predicted_words = []
                for i, (utt_id, orig_text, hyps) in enumerate(zip(batch.utt_id, orig_transcriptions, predicted_ids)):
                    print(f"\nsample {i+1} - ({utt_id}: '{orig_text}')")
                    sample_cers = []
                    for j, hyp in enumerate(hyps):
                        words = asr_brain.hparams.tokenizer.decode_ids(hyp.tokens.tolist()) # .split("|")
                        # words = tokenizer.decode_ids(hyp.tokens.tolist()) # .split("|")
                        hyp_cer = 100 * cer(orig_text, words)
                        sample_cers.append(hyp_cer)
                        print(f"\thyp {j+1}/{len(hyps)} (CER={hyp_cer:.1f}%): '{words}'")
                        predicted_words.append((f"sample {i+1}, hyp {j+1}/{len(hyps)}", words))
                        
                    print(f"\t=== Mean CER: {np.mean(sample_cers):.1f}%, Std CER: {np.std(sample_cers):.1f}% ===")

            transcripts.append(predicted_words)

    return transcripts, ctc_probs

# transcripts, ctc_probs = transcribe_dataset(asr_brain, test_set, greedy=False, num_batches_to_transcribe=1)

# LOAD WORD ALIGNED WAVS into dataset

In [28]:
# imitate CLAs
import sys
import argparse
import math
import glob
from tqdm import tqdm
import json

In [29]:
# set these in yaml config!

# these are to be loaded!
train_annotation_path = '/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/data/respeller_train_wordtoken_wav_annotation.json'
valid_annotation_path = '/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/data/respeller_valid_wordtoken_wav_annotation.json'
test_annotation_path = '/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/data/respeller_test_wordtoken_wav_annotation.json'

# this will be saved!
allsplits_annotation_path = '/home/s1785140/speechbrain/templates/speech_recognition_CharTokens_NoLM/data/respeller_allsplits_wordtoken_wav_annotation.json'

In [30]:
# create a new "combined annotation", that includes all of train, valid and test
def load_json(p):
    with open(p) as f:
        d = json.load(f)
    return d

allsplits_d = {}
for p in [train_annotation_path, valid_annotation_path, test_annotation_path]:
    allsplits_d.update(load_json(p))
    
json_data = json.dumps(allsplits_d)
with open(allsplits_annotation_path, 'w') as f:
    f.write(json_data)

In [31]:
speechbrain_hparams['train_annotation'] = train_annotation_path
speechbrain_hparams['valid_annotation'] = valid_annotation_path
speechbrain_hparams['test_annotation'] = test_annotation_path
speechbrain_hparams['allsplits_annotation'] = allsplits_annotation_path

In [32]:
def dataio_prepare(
    hparams, 
    split2data=None # use to optionally initialise datasets from data dicts rather than json files
):
    """This function prepares the datasets to be used in the brain class.
    It also defines the data processing pipeline through user-defined functions.


    Arguments
    ---------
    hparams : dict
        This dictionary is loaded from the `train.yaml` file, and it includes
        all the hyperparameters needed for dataset construction and loading.

    Returns
    -------
    datasets : dict
        Dictionary containing "train", "valid", and "test" keys that correspond
        to the DynamicItemDataset objects.
    """
    # Define audio pipeline. In this case, we simply read the path contained
    # in the variable wav with the audio reader.
    # wav path like: data/ljspeech_wavs_16khz_word_aligned/differs/differs__LJ001-0001__occ1__len8320.wav
    @sb.utils.data_pipeline.takes("wav")
    @sb.utils.data_pipeline.provides("sig", "wav_path", "utt_id")
    def audio_pipeline(wav_path):
        """Load the audio signal. This is done on the CPU in the `collate_fn`."""
        sig = sb.dataio.dataio.read_audio(wav_path)
        yield sig

        yield wav_path

        utt_id = wav_path.split("/")[-1].split(".")[0]
        yield utt_id

    @sb.utils.data_pipeline.takes("samples_to_graphemes_ratio")
    @sb.utils.data_pipeline.provides("samples_to_graphemes_ratio")
    def ratio_pipeline(samples_to_graphemes_ratio):
        yield samples_to_graphemes_ratio

    @sb.utils.data_pipeline.takes("length")
    @sb.utils.data_pipeline.provides("length")
    def length_pipeline(length):
        yield length

    @sb.utils.data_pipeline.takes("words")
    @sb.utils.data_pipeline.provides("words")
    def text_pipeline(words):
        """Processes the transcriptions to generate proper labels

        NB Make sure that you yield exactly what is defined above in @sb.utils.data_pipeline.provides()"""
        yield words

    # Define datasets from json data manifest file
    # Define datasets sorted by ascending lengths for efficiency
    datasets = {}

    load_annotations_from_json_files = split2data is None
    if load_annotations_from_json_files:
        # by default load all splits from the json files specified in hparams
        data_info = {
            "train": hparams["train_annotation"],
            "valid": hparams["valid_annotation"],
            "test": hparams["test_annotation"],
            "allsplits": hparams["allsplits_annotation"],
        }
    else:
        data_info = split2data
        
    for split in data_info:
        print(f"create Dataset from split {split}")
        if load_annotations_from_json_files:
            constructor = sb.dataio.dataset.DynamicItemDataset.from_json
        else:
            constructor = sb.dataio.dataset.DynamicItemDataset
            
        datasets[split] = constructor(
            data_info[split],
            dynamic_items=[audio_pipeline, ratio_pipeline, length_pipeline, text_pipeline],
            output_keys = [
                "id",
                "sig",
                "wav_path",
                "utt_id",
                "samples_to_graphemes_ratio",
                "length",
                "words",
            ],
        )
        hparams[f"{split}_dataloader_opts"]["shuffle"] = True
        datasets[split].split = split # add attribute to keep track of the split of the dataset

    def print_dataset_lens(extra_str):
        for split in data_info:
            dataset_split = datasets[split]
            print(f"{split} dataset has {len(dataset_split)} samples", extra_str)

    print_dataset_lens("before any filtering")

    # Filter data for samples_to_graphemes_ratio that is either too small or too large
    key_min_value = {}
    if hparams["min_samples_to_graphemes_ratio"] is not None:
        key_min_value = {"samples_to_graphemes_ratio": hparams["min_samples_to_graphemes_ratio"]}

    key_max_value = {}
    if hparams["max_samples_to_graphemes_ratio"] is not None:
        key_max_value = {"samples_to_graphemes_ratio": hparams["max_samples_to_graphemes_ratio"]}

    for split in data_info:
        datasets[split] = datasets[split].filtered_sorted(
            key_min_value=key_min_value,
            key_max_value=key_max_value,
        )
    print_dataset_lens("after filtering by min and max samples to graphemes ratio")

    # Filter samples whos length is too short
    key_min_value = {}
    if hparams["min_length_seconds"] is not None:
        key_min_value = {"length": hparams["min_length_seconds"]}
    for split in data_info:
        datasets[split] = datasets[split].filtered_sorted(
            key_min_value=key_min_value,
        )
    print_dataset_lens("after filtering by minimum length")

    # Sorting training data with ascending order makes the code  much
    # faster  because we minimize zero-padding. In most of the cases, this
    # does not harm the performance.
    if load_annotations_from_json_files:
        if hparams["sorting"] == "ascending":
            datasets["train"] = datasets["train"].filtered_sorted(sort_key="length")
            hparams["train_dataloader_opts"]["shuffle"] = False

        elif hparams["sorting"] == "descending":
            datasets["train"] = datasets["train"].filtered_sorted(
                sort_key="length", reverse=True
            )
            hparams["train_dataloader_opts"]["shuffle"] = False

        elif hparams["sorting"] == "random":
            hparams["train_dataloader_opts"]["shuffle"] = True
            pass

        else:
            raise NotImplementedError(
                "sorting must be random, ascending or descending"
            )
    
    return datasets

In [35]:
## filter dataset by PER of wordtypes as predicted by RNN-based G2P model
def filter_data_by_per(data, str_min_len_threshold, N):
    p = '/home/s1785140/data/ljspeech_fastpitch/oov_list_with_PER.pickle'
    loaded_df = pd.read_pickle(p)
    loaded_df = loaded_df[loaded_df['wordtype'].str.len() >= str_min_len_threshold]
    loaded_df = loaded_df.sort_values('per', ascending=False).head(N)
    wordtypes_to_keep = set(loaded_df['wordtype'])
    new_data = {}
    for token_id, token_id_values in data.items():
        wordtype = token_id_values['words']
        if wordtype not in wordtypes_to_keep:
            continue
        new_data[token_id] = token_id_values
    return new_data

filter_by_per = True
# load from json files
if not filter_by_per:
    datasets = dataio_prepare(speechbrain_hparams)
else:
    filtered_allsplits_d = filter_data_by_per(allsplits_d, str_min_len_threshold=10, N=50)
    datasets = dataio_prepare(speechbrain_hparams, {'allsplits': filtered_allsplits_d})

create Dataset from split allsplits
allsplits dataset has 66 samples before any filtering
allsplits dataset has 66 samples after filtering by min and max samples to graphemes ratio
allsplits dataset has 66 samples after filtering by minimum length


In [36]:
# convert from datasets to dataloaders
split2stage = {"train": sb.Stage.TRAIN, "valid": sb.Stage.VALID, "test": sb.Stage.TEST, "allsplits": sb.Stage.TEST}
for split in datasets.keys():
    if not isinstance(datasets[split], DataLoader) or isinstance(datasets[split], LoopedLoader):
        dataloader_kwargs=speechbrain_hparams[f"{split}_dataloader_opts"]
        datasets[split] = asr_brain.make_dataloader(
            datasets[split], stage=split2stage[split], **dataloader_kwargs
        )

In [37]:
def set_whitespace_to_0_probability(ctc_probs, vocab, log_probs=True, whitespace_symbol="|"):
    """ctc_probs [bsz, max_seq_len, vocab_size]"""
    new_probability = -math.inf if log_probs else 0.0
    ctc_probs[:,:,vocab.index(whitespace_symbol)] = new_probability
    return ctc_probs

# TRANSCRIBE WORD ALIGNED WAVS

In [38]:
from collections import defaultdict

# generate transcriptions for all batches in test set
def transcribe_dataset(asr_brain, dataset, 
                       num_batches_to_transcribe=None,
                       hack_whitespace_probs=False, collapse_whitespace=True,
                       print_info=False, max_hyps_per_sample=None,
                       wordtypes_to_transcribe=[]):
    # Now we iterate over the dataset and we simply compute_forward and decode

    orig_words = []
    printouts = []
    orig_wavs = []
    token_ids = []
    transcribed_words = defaultdict(list)
    n = 0 # number of batches transcribed
    
    break_forloop_early = num_batches_to_transcribe is not None

    with torch.no_grad():
        for batch in tqdm(dataset, dynamic_ncols=True, total=num_batches_to_transcribe):
            # break out of loop if we have transcribed enough batches
            if break_forloop_early and n >= num_batches_to_transcribe:
                break
            n += 1

            orig_transcriptions = batch.words
            wavs = batch.sig.data

            # Make sure that your compute_forward returns the predictions !!!
            # In the case of the template, when stage = TEST, a beam search is applied 
            # in compute_forward(). 
            predictions = asr_brain.compute_forward(batch, stage=sb.Stage.TEST)
            ctc_probs = predictions['ctc_logprobs'] # FOR DEBUG

            # hack probabilities to set all probs to 0 for whitespace
            if hack_whitespace_probs:
                ctc_probs = set_whitespace_to_0_probability(ctc_probs, vocab, log_probs=True, whitespace_symbol="|")

            # get mel lens from wav len ratios since torch ctc decoder requires lens in frames
            batch_max_len = predictions["ctc_logprobs"].size(1)
            bsz = predictions["ctc_logprobs"].size(0)
            mel_lens = torch.zeros(bsz)
            for i, len_ratio in enumerate(asr_brain.feat_lens):
                mel_lens[i] = int(torch.round(len_ratio * batch_max_len))
            
            predicted_ids = ctc_beamsearch_decoder(
                predictions["ctc_logprobs"], lengths=mel_lens
            )

            # iterate over samples in batch
            for i, (utt_id, orig_text, hyps, wav) in enumerate(zip(batch.utt_id, orig_transcriptions, predicted_ids, wavs)):
                sample_printout = f"\nsample {i+1} - ({utt_id}: '{orig_text}')"
                orig_utt_text = "INSERT ORIG UTT TEXT"
                sample_printout += f"\nOriginal Utterance: {orig_utt_text}"
                # if print_info: print(f"\nsample {i+1} - ({utt_id}: '{orig_text}')")
                sample_cers = []
                for j, hyp in enumerate(list(hyps)[:max_hyps_per_sample]):
                    words = asr_brain.hparams.tokenizer.decode_ids(hyp.tokens.tolist())
                    if collapse_whitespace:
                        words = "".join(c for c in words if c != " ")
                    hyp_cer = 100 * cer(orig_text, words)
                    sample_cers.append(hyp_cer)
                    sample_printout += f"\n\thyp {j+1}/{len(hyps)} (CER={hyp_cer:.1f}%): '{words}'"
                    # if print_info: print(f"\thyp {j+1}/{len(hyps)} (CER={hyp_cer:.1f}%): '{words}'")
                    if not wordtypes_to_transcribe or orig_text in wordtypes_to_transcribe:
                        transcribed_words[orig_text].append(words)
                    
                sample_printout += f"\n\t=== Mean CER: {np.mean(sample_cers):.1f}%, Std CER: {np.std(sample_cers):.1f}% ==="
                # if print_info: print(f"\t=== Mean CER: {np.mean(sample_cers):.1f}%, Std CER: {np.std(sample_cers):.1f}% ===")

                include_current_wordtype = (wordtypes_to_transcribe == [] or orig_text in wordtypes_to_transcribe)
                if include_current_wordtype:
                    orig_words.append(orig_text)
                    printouts.append(sample_printout)
                    orig_wavs.append(wav)
                    token_ids.append(utt_id)

    output_dict = {
        "orig_words": orig_words,
        "transcribed_words": transcribed_words,
        "wavs": orig_wavs,
        "printouts": printouts,
        "token_ids": token_ids,
    }

    return output_dict


## Generate multiple times to see if outputs change (i.e. should change if dropout is enabled in model)

In [ ]:
# NUM_BATCHES_TO_TRANSCRIBE = 1
NUM_BATCHES_TO_TRANSCRIBE = None
MAX_HYPS_PER_SAMPLE = 50

wordtypes_to_transcribe = []
# wordtypes_to_transcribe = ['anesthesiologists'] # used for debugging and checking results only for a subset of wordtypes

NUM_ITERATIONS = 25

word_sets = defaultdict(set)

print_printout = False
display_audio = False

'''create output dict of form:
token_id:
    word: xxx
    hyps:
        xxx
        xxx
        xxx
        ...
    wav: /path/to/token_id.wav
'''
master_transcription_dict = defaultdict(dict)
print_num_new_spellings_per_iteration = False

for i in range(NUM_ITERATIONS):
    print(f"Iteration {1+i}/{NUM_ITERATIONS}")
    transcription_output_dict = transcribe_dataset(asr_brain, datasets["allsplits"], 
                                                num_batches_to_transcribe=NUM_BATCHES_TO_TRANSCRIBE,
                                                collapse_whitespace=False, hack_whitespace_probs=True,
                                                max_hyps_per_sample=MAX_HYPS_PER_SAMPLE,
                                                wordtypes_to_transcribe=wordtypes_to_transcribe)
    
    for token_id, orig_word, printout, wav in zip(
        transcription_output_dict["token_ids"], 
        transcription_output_dict["orig_words"], 
        transcription_output_dict["printouts"], 
        transcription_output_dict["wavs"]
    ):
        if print_printout:
            print(printout)
        if display_audio:
            display(Audio(wav, rate=16000))
        old_count = len(word_sets[orig_word])
        asr_hypotheses = transcription_output_dict["transcribed_words"][orig_word]
        asr_hypotheses = [hyp.rstrip() for hyp in asr_hypotheses] # as CTC decoder always adds extra whitespace to end
        word_sets[orig_word].update(asr_hypotheses)
        master_transcription_dict[token_id]["orig_word"] = orig_word
        master_transcription_dict[token_id]["orig_wav"] = wav.numpy()             
        
        if "hyps" not in master_transcription_dict[token_id]:
            master_transcription_dict[token_id]["hyps"] = set(asr_hypotheses)
        else:
            master_transcription_dict[token_id]["hyps"].update(asr_hypotheses)
        
        new_count = len(word_sets[orig_word])
        if new_count > old_count:
            if print_num_new_spellings_per_iteration:
                print(f"added {new_count - old_count} new spellings for '{orig_word}' to its word set")

total_num = 0
for orig_word, word_set in word_sets.items():
    print(f"added {len(word_set)} new spellings for '{orig_word}' to its word set")
    total_num += len(word_set)
    
print(f"\n *** Will synthesise these {total_num} spellings using TTS pipeline")

Iteration 1/25


100%|███████████████████████████████████████████████████████| 2/2 [00:27<00:00, 13.68s/it]


Iteration 2/25


  0%|                                                               | 0/2 [00:00<?, ?it/s]

# create dataframe to store respellings for each original spelling (and respective CERs)

In [ ]:
# create big dataframe to hold all data
df = pd.DataFrame(columns=['token_id', 'wordtype', 'respelling', 'cer'])
data = [] # used to populate pandas dataframe

for token_id in master_transcription_dict:    
    for hyp in master_transcription_dict[token_id]['hyps']:
        data.append({'token_id': token_id, 
                     'wordtype': master_transcription_dict[token_id]['orig_word'], 
                     'respelling': hyp, 
                     'orig_wav': master_transcription_dict[token_id]['orig_wav'],
                     'cer': 100*cer(master_transcription_dict[token_id]['orig_word'], hyp)}),

df = pd.concat([df, pd.DataFrame(data)])
print('number of token_id-spelling combinations', len(df))

In [ ]:
# guarantee that original spelling will be generated by TTS
# i.e. add orig_word to "respelling" column if it doesn't occur in it already
dfs_to_concat = []
for orig_word in df['wordtype'].unique():
    match_mask = df['respelling'] == orig_word
    matching_respellings = df.loc[match_mask]
    if matching_respellings.empty:
        # copy the first row, but change value for respelling
        first_row_df = df.loc[df['wordtype'] == orig_word].head(1)
        row_idx = first_row_df.index[0]
        first_row_df.loc[row_idx, 'respelling'] = orig_word
        first_row_df.loc[row_idx, 'cer'] = 0.0
        dfs_to_concat.append(first_row_df)
        # print(f'added original spelling to respelling column for "{orig_word}"')

for temp_df in dfs_to_concat:
    df = pd.concat([df, temp_df], ignore_index=True)
print('number of token_id-spelling combinations', len(df))

# load pretrained fastpitch and hifigan models

In [ ]:
from load_pretrained_tts_and_vocoder import load_vocoder, load_pretrained_fastpitch

In [ ]:
hifigan_checkpoint = "/home/s1785140/pretrained_models/hifigan/ljspeech/LJ_V1/generator_v1"
fastpitch_checkpoint = '/home/s1785140/respeller/fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt'

In [ ]:
hifigan = load_vocoder(hifigan_checkpoint)

In [ ]:
fastpitch, *_ = load_pretrained_fastpitch(fastpitch_checkpoint)

# run each respelling through TTS to get mel-spectrogram (and optionally vocode to get wav)

In [ ]:
from fastpitch.inference import prepare_input_sequence

In [ ]:
# create input to fastpitch batch maker
all_orig_words = [orig_word for orig_word in df['wordtype']]
all_respellings = [respelling for respelling in df['respelling']]
fields = {'orig_spelling': all_orig_words, 
          'respelling': all_respellings,
          'text': all_respellings} # will be encoded by text processes into IDs

In [ ]:
batches = prepare_input_sequence(fields, device, input_type='char', 
                       symbol_set='english_pad_lowercase_nopunc', 
                       text_cleaners=['lowercase_no_punc'],
                       batch_size=32)

In [ ]:
def melspec2mfcc(mel, n_mfcc=12):
    log_mel = librosa.core.amplitude_to_db(mel, ref=np.max)
    mfcc = librosa.feature.mfcc(S=log_mel, n_mfcc=n_mfcc)
    return mfcc

def generate_audio(y, vocoder=None, sampling_rate=22050, hop_length=256):
    """Generate audio from spectrograms for n utterances in batch"""
    mel, mel_lens = y
    bs = mel.size(0)
    with torch.no_grad():
        audios = vocoder(mel).cpu().squeeze(1).numpy() # [bsz, dim, samples ]only squeeze away dim (equals 1 for waveform)
        mel_lens = mel_lens.cpu().numpy() - 1
    audios_to_return = []
    for audio, mel_len in zip(audios, mel_lens):
        audio = audio[:mel_len * hop_length]
        audio = audio / np.max(np.abs(audio))
        audios_to_return.append(audio)
        
    return audios_to_return

In [ ]:
do_vocode = True

spelling2tts_mel = {}
spelling2tts_mfcc = {}
spelling2vocoded_audio = {}
audios = None

with torch.no_grad():
    for b in tqdm(batches, "synthesising respellings"):
        mel, mel_lens, *_ = fastpitch(b['text'])
        
        if do_vocode:
            vocoder_inputs = (mel, mel_lens)
            audios = generate_audio(vocoder_inputs, hifigan)
            
        # add samples in batch to dicts
        bsz = len(mel)
        for i in range(bsz):
            respelling = b['respelling'][i]
            mel_len = mel_lens[i]
            spelling2tts_mel[respelling] = mel[i, :, :mel_len].cpu().numpy() # mel [bsz, feats, time]
            spelling2tts_mfcc[respelling] = melspec2mfcc(spelling2tts_mel[respelling])
            
            if audios is not None:
                spelling2vocoded_audio[respelling] = audios[i]

# create mapping from token_id to ground truth fastpitch mels 

In [ ]:
#build mapping from token_id to path to its gt word aligned mel 
word_aligned_feats_path = '/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels_lowercase_nopunc'
token_id2fastpitch_gt_mel_path = {}

for token_id in df['token_id'].unique():
    wordtype = token_id.split('__')[0]
    path = os.path.join(word_aligned_feats_path, wordtype, token_id + '.pt')
    token_id2fastpitch_gt_mel_path[token_id] = path

In [ ]:
# load mels from disk
token_id2fastpitch_gt_mel = {}

for token_id, gt_mel_path in token_id2fastpitch_gt_mel_path.items():
    # remove len and file extension from path
    # e.g. 'wordaligned_mels/unconstitutionality/unconstitutionality__LJ021-0191__occ1__len25760.wav' 
    # ---> 'wordaligned_mels/unconstitutionality/unconstitutionality__LJ021-0191__occ1"
    path_no_len_no_ext = gt_mel_path.split('__len')[0]
    
    # glob to fine the matching ground truth mel saved on disk
    matches = glob.glob(path_no_len_no_ext + '*.pt')
    assert len(matches) == 1
    fastpitch_gt_mel = torch.load(matches[0]).transpose(0,1).cpu().numpy()
    token_id2fastpitch_gt_mel[token_id] = fastpitch_gt_mel
    

# Verify that predicted mel-spectrograms are normalised in same way as GT mel-spectrograms

In [ ]:
def plot_melspec(mel, title=""):
    if type(mel) == torch.Tensor:
        mel = mel.cpu().numpy()
    fig, ax = plt.subplots(figsize=(10, 2))
    if title:
        plt.title(title)
    plt.imshow(mel, aspect='auto', origin='lower', interpolation='none')
    plt.colorbar()
    
def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0, constant_values=0) -> np.ndarray:
    # Pad the data with NaN values at both ends
    pad_size = target_length - array.shape[axis]
    assert pad_size % 2 == 0

    if pad_size <= 0:
        return array

    npad = [(0, 0)] * array.ndim
    npad[axis] = (int(pad_size/2), int(pad_size/2))

    return np.pad(array, pad_width=npad, mode='constant', constant_values=constant_values)

def wma_smooth(mel, weights, dim_to_smooth=0, maintain_time_dim=False, extra_padding=0):
    # mel should be shape [t, dim]
    if dim_to_smooth == 0:
        data = mel.transpose(0,1)
    elif dim_to_smooth == 1:
        data = mel
    else:
        raise ValueError

    weights = weights / np.sum(weights)
    window = weights.shape[0]
    dim = data.shape[1]
        
    # stack along feat dimension
    stacked_weights = [weights for _ in range(dim)]
    stacked_weights = np.stack(stacked_weights, axis=-1)
    weights = stacked_weights

    # pad data
    target_len_after_padding = data.shape[0] + 2 * (window // 2) + 2 * extra_padding
    padded_data = pad_along_axis(data, 
                                 target_length=target_len_after_padding, 
                                 axis=0, 
                                 constant_values=data.min()) # use data.min() as padding val because for log mel, 0 is actually large!!!

    # Compute the weighted moving average 
    N = len(padded_data) - window + 1
    wma = np.array([np.sum(weights * padded_data[i:i+window], axis=0) / np.sum(np.nan_to_num(weights), axis=0) for i in range(N)])
    
    if maintain_time_dim:
        assert wma.shape == data.shape, f"{wma.shape=} == {data.shape=}"
    else:
        assert wma.shape[1] == data.shape[1], f"{wma.shape[1]=} == {data.shape[1]=}"
    
    return wma

In [ ]:
# plot GT mel spectrogram
token_id = list(token_id2fastpitch_gt_mel.keys())[0]
wordtype = token_id.split('__')[0]
gt_mel = token_id2fastpitch_gt_mel[token_id]

plot_melspec(gt_mel, token_id + " - ORIGINAL!!! (no weighted moving average)")

In [ ]:
filter = [1,2,1]
wma = wma_smooth(gt_mel.T, weights=filter).T
plot_melspec(wma, token_id + f" weights={filter}")

In [ ]:
temporal_filter = [1,2,1]
feature_filter = [1,1,2,1,1]
wma = wma_smooth(gt_mel.T, dim_to_smooth=0, weights=temporal_filter).T
wma = wma_smooth(wma.T, dim_to_smooth=1, weights=feature_filter).T
plot_melspec(wma, token_id + f" weights={filter}")

In [ ]:
# plot predicted mel spectrogram
mel = spelling2tts_mel[wordtype]

print(wordtype)
plot_melspec(mel, wordtype)

# convert mel spec to Mel-Cepstrum then plot

In [ ]:
def plot_mfcc(mfcc, extra_title=""):
    # assume mfcc is a numpy array of shape (n_mfcc, n_frames)
    plt.figure(figsize=(10, 4))
    plt.imshow(mfcc, aspect='auto', origin='lower', cmap='coolwarm')
    plt.xlabel('Time')
    plt.ylabel('MFCC')
    title = 'MFCC'
    if extra_title:
        title += f": {extra_title}"
    plt.title(title)
    plt.colorbar()
    plt.tight_layout()
    plt.show()

In [ ]:
mfcc = melspec2mfcc(mel)
plot_mfcc(mfcc, wordtype)

In [ ]:
gt_mfcc = melspec2mfcc(gt_mel)
plot_mfcc(gt_mfcc, token_id)

# (optional) apply weighted moving average filters to GT mels

In [ ]:
# temporal_filter = []
# feature_filter = []
temporal_filter = [1,2,1]
feature_filter = [1,1,2,1,1]

for token_id, fastpitch_gt_mel in token_id2fastpitch_gt_mel.items():
    if temporal_filter:
        fastpitch_gt_mel = wma_smooth(fastpitch_gt_mel.T, dim_to_smooth=0, weights=temporal_filter).T
    if feature_filter:
        fastpitch_gt_mel = wma_smooth(fastpitch_gt_mel.T, dim_to_smooth=1, weights=feature_filter).T       

# convert GT mels to MFCCs

In [ ]:
token_id2fastpitch_gt_mfcc = {}
for token_id, fastpitch_gt_mel in token_id2fastpitch_gt_mel.items():
    token_id2fastpitch_gt_mfcc[token_id] = melspec2mfcc(fastpitch_gt_mel)  

# add acoustic distances between ground truth audio for word and synthesised respellings

compare ranking by:
- MCD-DTW
- SoftDTW
- L1, L2 distance

In [ ]:
import mcd
from mcd import dtw

def eucCost(x, y):
    """AKA l2 norm"""
    return math.sqrt(np.inner(x - y, x - y))

def manCost(x, y):
    """AKA l1 norm"""
    return np.sum(np.inner(x - y, x - y))

metric_registry = {
}

metric_type = "mcd_dtw"

distance2costfn = {
    "euclidean": eucCost,
    "manhattan": manCost,
}

distance_type = "euclidean"

distance_metric_col_name = f'{metric_type} ({distance_type})'

In [ ]:
# calculate MCD-DTW distance for each synthesised respelling in the dataframe
# to the gt mfcc for the token_id that the respelling was generated from (using ASR)
print(f"calculating MCD-DTW distances between GT and predicted mels using {distance_type}")

for row_idx in tqdm(df.index):
    respelling_mfcc = spelling2tts_mfcc[df.loc[row_idx, 'respelling']]
    gt_mfcc = token_id2fastpitch_gt_mfcc[df.loc[row_idx, 'token_id']]
    respelling_mfcc = respelling_mfcc.T
    gt_mfcc = gt_mfcc.T
    distance_score, _path = dtw.dtw(gt_mfcc, respelling_mfcc, distance2costfn[distance_type])
    df.loc[row_idx, distance_metric_col_name] = distance_score

In [ ]:
# sort df by distance then spelling
df = df.sort_values(['wordtype', distance_metric_col_name])

# play audios for each word, ranked by certain metric (e.g. CER, acoustic distance, etc.)

In [ ]:
wordtypes = df["wordtype"].unique()
print(len(wordtypes))
wordtypes

In [ ]:
# optionally choose to only display a subset of the wordtypes

words_to_include = None # include all words
# words_to_include = [
#     "presenting",
#     "admittedly",
# ]

if words_to_include is not None:
    filtered_df = pd.DataFrame()
    for wordtype in words_to_include:
        filtered_df = pd.concat([filtered_df, df.loc[df["wordtype"] == wordtype]]) 
else:
    filtered_df = df

In [ ]:
# Optionally only display N-best ranked respellings per original input audio (excluding original spellings!)
get_smallest = True
n_spellings_per_token_id = 5
if get_smallest:
    lambda_fn = lambda x: x.nsmallest(n_spellings_per_token_id, distance_metric_col_name)
else:
    lambda_fn = lambda x: x.nlargest(n_spellings_per_token_id, distance_metric_col_name)
    
original_spellings_mask = filtered_df['wordtype'] == filtered_df['respelling']
original_spellings_df = filtered_df[original_spellings_mask]
inverted_mask = ~original_spellings_mask
no_orig_spellings_df = filtered_df[inverted_mask]
filtered_df = no_orig_spellings_df.groupby('wordtype').apply(lambda_fn)

In [ ]:
# add back original spellings and sort by distance again
filtered_df = pd.concat([filtered_df, original_spellings_df], ignore_index=True)
filtered_df = filtered_df.groupby('wordtype', group_keys=True).apply(lambda x: x.sort_values(distance_metric_col_name))

In [ ]:
# add column to indicate whether it is original spelling or not
new_col_idx = 1 + filtered_df.columns.get_loc('respelling') # insert after respelling column
filtered_df.insert(new_col_idx, 'is_orig', filtered_df['wordtype'] == filtered_df['respelling'])

In [ ]:
current_wordtype = ""
for token_id, wordtype, respelling, distance, orig_wav, is_orig in zip(filtered_df['token_id'], 
                                                              filtered_df['wordtype'], 
                                                              filtered_df['respelling'], 
                                                              filtered_df[distance_metric_col_name], 
                                                              filtered_df['orig_wav'],
                                                              filtered_df['is_orig']):
    # new wordtype, print GT token_ids and display audios
    if current_wordtype != wordtype:
        current_wordtype = wordtype
        
        # get all token_ids and orig_wavs for this wordtype
        wordtype_df = filtered_df[filtered_df['wordtype'] == wordtype]
        first_row_for_token_id = wordtype_df.groupby('token_id').nth(0)
        
        print("\n===============================================================")
        print(f"====== Original word aligned WAV(s) ===========================")
        print("===============================================================")
        for index, row in first_row_for_token_id.iterrows():
            token_id = index # index is now equal to token_id as we used groupby above
            print(f"[GT - {token_id}]:")
            display(Audio(row['orig_wav'], rate=16000))
        print("===============================================================")
    
    # create string to print for this respelling
    print_str = ""
    if is_orig:
        print_str += f"{' '+wordtype+' ':*^35}"
    else:
        print_str += f" ---> {respelling=}"
    distance_str = f" {distance:.2f} [{distance_metric_col_name}] [calc'ed vs GT: {token_id}]"
    print_str += distance_str
    print(print_str)
    
    display(Audio(spelling2vocoded_audio[respelling], rate=22050))